In [70]:
import requests
import time
import json
from bs4 import BeautifulSoup
from stackapi import StackAPI
from tqdm import tqdm

In [144]:
models = ["alpaca", "bloom", "cerebras-gpt", "chatglm", "chinchilla", "codex", "codegen", "codegx", "dolly-v2", "eleuther-pythia", "falcon", "fastchat-t5", "gal", "gpt-3", "gpt-3.5", "gpt-4", "gpt4all", "gpt-neox", "gpt-j", "koala", "llama", "mpt", "oasst-pythia", "opt", "palm", "palm-coder", "replit-code-v1", "stablelm-base-alpha", "stablelm-tuned-alpha", "starcoder-base", "starcoder", "vicuna", "llama-2"]
topics = ["large-language-models", "llm", "word-embedding", "gpt", "intelligent-agent"]
tags = models + topics


In [145]:
SITE = StackAPI('ai', key="GXK03oiqVULDxPyWtLmyNA((")
q_blocks = {}
for tag in tqdm(tags):
    questions = SITE.fetch('questions', tagged=tag, sort = 'votes', filter='withbody')
    
    for question in questions['items']:
        content = BeautifulSoup(question['body']).get_text()
        q_blocks[question['question_id']] = {"link": question['link'], "metadata": question, "title": question["title"], "content": content, "question_id": question['question_id']}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:13<00:00,  2.77it/s]


In [146]:
SITE = StackAPI('genai', key="GXK03oiqVULDxPyWtLmyNA((")
questions = SITE.fetch('questions', sort = 'votes', filter='withbody')
for question in questions['items']:
    content = BeautifulSoup(question['body']).get_text()
    q_blocks[question['question_id']] = {"link": question['link'], "metadata": question, "title": question["title"], "content": content, "question_id": question['question_id']}


In [149]:
q_blocks

KeyError: 0

In [150]:
for question_id, q_block in tqdm(q_blocks.items()):
    answers = SITE.fetch('questions/{ids}/answers', ids = [question_id], sort = 'votes', filter='withbody')
    
    # Find top voted answers
    if q_block.get("answers") is None:
        q_block['answers'] = []
    for item in answers['items']:
        body = BeautifulSoup(item['body']).get_text()
        q_block['answers'].append({"answer_id": item['answer_id'], "body": body, "score": item['score'], "is_accepted": item["is_accepted"]}) 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 443/443 [02:24<00:00,  3.06it/s]


In [181]:
filtered = {}
for idx, (qid, q_block) in enumerate(q_blocks.items()):
    keywords = ["chatgpt", "image", "stable diffusion", "draw", "deep-RL", ".net", "RL", "nasty", "AUTOMATIC1111"]
    skip = False
    for keyword in keywords:
        if keyword.lower() in q_block['title'].lower() or keyword.lower() in q_block['content'].lower():
            skip = True
    if skip:
        continue
    print(f"title: {q_block['title']}\n")
    print(f"content: {q_block['content'][:250]}...\n")
        # print(f"answers: {q_block['answers']}")
    filtered[qid] = q_block

title: What is the &quot;temperature&quot; in the GPT models?

content: What does the temperature parameter mean when talking about the GPT models?
I know that a higher temperature value means more randomness, but I want to know how randomness is introduced.
Does temperature mean we add noise to the weights/activations o...

title: What language is the GPT-3 engine written in?

content: I know that the API is python based, but what's the gpt-3 engine written in mostly? C? C++? I'm having some trouble finding this info.
...

title: Is GPT-4 based on GPT-3 or was it trained from the scratch?

content: To me it looks like GPT-4 is based on GPT-3.
On the other hand, there were rumors that training of GPT-3 was done with errors, but re-train was impossible due to the costs.
...

title: How to get GPT-3 to translate a specific word in a sentence?

content: I just gave GPT-3 the following prompt (in the playground, using text-davinci-001 with default settings):
What's the German word for "can

In [178]:
json.dump(filtered, open("aistackexchange.json", "w"))